In [1]:
import numpy as np
import cvxpy as cp
import heapdict
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import axes3d, Axes3D

import time

In [2]:
#https://github.com/ClayFlannigan/icp/blob/master/icp.py

def best_fit_transform(A, B):
    '''
    Calculates the least-squares best-fit transform that maps corresponding points A to B in m spatial dimensions
    Input:
      A: Nxm numpy array of corresponding points
      B: Nxm numpy array of corresponding points
    Returns:
      T: (m+1)x(m+1) homogeneous transformation matrix that maps A on to B
      R: mxm rotation matrix
      t: mx1 translation vector
    '''


    # get number of dimensions
    m = A.shape[1]

    # translate points to their centroids
    centroid_A = np.mean(A, axis=0)
    centroid_B = np.mean(B, axis=0)
    AA = A - centroid_A
    BB = B - centroid_B

    # rotation matrix
    H = np.dot(AA.T, BB)
    U, S, Vt = np.linalg.svd(H)
    R = np.dot(Vt.T, U.T)

    # special reflection case
    if np.linalg.det(R) < 0:
       Vt[m-1,:] *= -1
       R = np.dot(Vt.T, U.T)

    # translation
    t = centroid_B.T - np.dot(R,centroid_A.T)

    # homogeneous transformation
    T = np.identity(m+1)
    T[:m, :m] = R
    T[:m, m] = t

    return T, R, t


def nearest_neighbor(src, dst):
    '''
    Find the nearest (Euclidean) neighbor in dst for each point in src
    Input:
        src: Nxm array of points
        dst: Nxm array of points
    Output:
        distances: Euclidean distances of the nearest neighbor
        indices: dst indices of the nearest neighbor
    '''

    assert src.shape == dst.shape

    neigh = NearestNeighbors(n_neighbors=1)
    neigh.fit(dst)
    distances, indices = neigh.kneighbors(src, return_distance=True)
    return distances.ravel(), indices.ravel()


def icp(A, B, init_pose=None, max_iterations=20, tolerance=0.001, C=[]):
    '''
    The Iterative Closest Point method: finds best-fit transform that maps points A on to points B
    Input:
        A: Nxm numpy array of source mD points
        B: Nxm numpy array of destination mD point
        init_pose: (m+1)x(m+1) homogeneous transformation
        max_iterations: exit algorithm after max_iterations
        tolerance: convergence criteria
    Output:
        T: final homogeneous transformation that maps A on to B
        distances: Euclidean distances (errors) of the nearest neighbor
        i: number of iterations to converge
    '''

    assert A.shape == B.shape
    
    # get number of dimensions
    m = B.shape[1]

    # make points homogeneous, copy them to maintain the originals
    src = np.ones((m+1,B.shape[0]))
    dst = np.ones((m+1,A.shape[0]))
    src[:m,:] = np.copy(B.T)
    dst[:m,:] = np.copy(A.T)

    # apply the initial pose estimation
    if init_pose is not None:
        src = np.dot(init_pose, src)

    prev_error = 0

    for i in range(max_iterations):
        # find the nearest neighbors between the current source and destination points
        distances, indices = nearest_neighbor(src[:m,:].T, dst[:m,:].T)
        
        for idx, corresp in C:
            indices[idx] = corresp

        # compute the transformation between the current source and nearest destination points
        T,_,_ = best_fit_transform(src[:m,:].T, dst[:m,indices].T)

        # update the current source
        src = np.dot(T, src)

        # check error
        mean_error = np.mean(distances)
        if np.abs(prev_error - mean_error) < tolerance:
            break
        prev_error = mean_error

    # calculate final transformation
    T,_,_ = best_fit_transform(B, src[:m,:].T)
    
    distances, indices = nearest_neighbor(B, src[:m,:].T)

    return T, distances, indices, i

In [49]:
def points_from_mask(B, D):
    minN = int(np.sum(D))
    Bt_map = np.argwhere(D==1).T[0].T
    Bt = np.zeros((minN,3))
    Bt[np.arange(minN)] = B[Bt_map]
    return Bt, Bt_map

def icp_bnb(A, B):
    
    An = A.shape[0]
    Bn = B.shape[0]
    
    #Currently under assumption we want to rotate B onto A, and Bn >= An
    #TODO: Change this assumption
    
    minN = min(An,Bn)
    
    D = np.zeros(Bn)
    diff = Bn - An
    D[:minN] = 1
    
    Bt, Bt_map = points_from_mask(B,D)
    
    hd = heapdict.heapdict()
    hd_size = 0
    
    T, distances, indices, i = icp(A,Bt)
    objective = np.sum(distances)
    
    hd[objective] = (T, distances, indices)
    hd_size += 1
    
    iters = 500
    prev_objective_global = 0 
    while(hd_size > 0 and iters > 0):
        
        objective_global, (T, distances, indices) = hd.popitem()
        hd_size -= 1
        
        distances, indices = compute_dist(A,B,T)
        objective_global = np.sum(distances)
        
        #only look at distances currently active in the mask
        n = max(25, 15*int(prev_objective_global - objective_global))
        if (prev_objective_global == 0):
            n = 100 #initial
        print('Replacing ' + str(n) + ' samples.')
        idx_remove = (np.multiply(D, distances)).argsort()[-n:]
        D_inv = D*99999 + 1
        idx_add = (np.multiply(D_inv, distances)).argsort()[:n]
        
        for i in idx_remove:
            assert D[i] == 1
        for i in idx_add:
            assert D[i] == 0
        
        D[idx_remove] = 0
        D[idx_add] = 1
        
        prev_objective_global = objective_global
        
        Bt, Bt_map = points_from_mask(B,D)
        
        T, distances, indices, i = icp(A,Bt)
        objective_local = np.sum(distances)
        
        distances, indices = compute_dist_BA(A,B,T)
        objective_global = np.sum(distances)
        
        print('{0:.3f}'.format(objective_local) + ' | ' + '{0:.3f}'.format(objective_global))
        
        hd[objective_global] = (T, distances, indices)
        hd_size += 1
        
        iters -= 1
        
        if(objective_local < 1127):
            return T, objective
    
    
    return T, objective
    

In [50]:
def loadPointCloud(filename):
    pcloud = np.loadtxt(filename, skiprows=1);
    return pcloud;

In [51]:
def transform(B,T):
    Bh = np.ones((4,B.shape[0]))
    Bh[:3,:] = np.copy(B.T)
    Bh = Bh.T
    Bh = Bh @ T

    Bh[:,0] /= Bh[:,3]
    Bh[:,1] /= Bh[:,3]
    Bh[:,2] /= Bh[:,3]
    Bh[:,3] /= Bh[:,3]
    
    return Bh[:,:3]

def compute_dist(A,B,T):
    Bt = B
    At = transform(A,T)
    N = Bt.shape[0]
    
    neigh = NearestNeighbors(n_neighbors=1)
    neigh.fit(At)
    distances, indices = neigh.kneighbors(Bt, return_distance=True)
    return distances.ravel(), indices.ravel()

def compute_dist_BA(A,B,T):
    Bt = B
    At = transform(A,T)
    N = Bt.shape[0]
    
    neigh = NearestNeighbors(n_neighbors=1)
    neigh.fit(Bt)
    distances, indices = neigh.kneighbors(At, return_distance=True)
    return distances.ravel(), indices.ravel()

In [52]:
def visualize(A,B,T):

    fig = plt.figure()
    ax = Axes3D(fig)

    Bh = np.ones((4,B.shape[0]))
    Bh[:3,:] = np.copy(B.T)
    Bh = Bh.T
    Bh = Bh @ T

    Bh[:,0] /= Bh[:,3]
    Bh[:,1] /= Bh[:,3]
    Bh[:,2] /= Bh[:,3]
    Bh[:,3] /= Bh[:,3]

    ax.scatter3D(A[:,0], A[:,1], A[:,2], c=A[:,2], cmap='Greens');
    ax.scatter3D(Bh[:,0], Bh[:,1], Bh[:,2], c=Bh[:,2], cmap='Reds');

In [53]:
start = time.time()

A = loadPointCloud("data/data_bunny.txt")
B = loadPointCloud("data/model_bunny.txt")

T, _ = icp_bnb(A, B)

k, i = compute_dist(A,B,T)
print(np.sum(k))
print("Local registration took %.3f sec.\n" % (time.time() - start))

Replacing 100 samples.
1635.338 | 4223.644
Replacing 25 samples.
1629.079 | 4223.338
Replacing 25 samples.
1622.840 | 4222.757
Replacing 25 samples.
1615.100 | 4221.545
Replacing 25 samples.
1608.331 | 4220.347
Replacing 25 samples.
1600.174 | 4219.026
Replacing 25 samples.
1593.572 | 4218.240
Replacing 25 samples.
1587.456 | 4217.577
Replacing 25 samples.
1579.644 | 4216.871
Replacing 25 samples.
1573.632 | 4216.059
Replacing 25 samples.
1567.448 | 4215.826
Replacing 25 samples.
1562.581 | 4214.304
Replacing 25 samples.
1555.382 | 4213.186
Replacing 25 samples.
1548.620 | 4211.844
Replacing 25 samples.
1542.985 | 4210.600
Replacing 25 samples.
1538.254 | 4209.835
Replacing 25 samples.
1531.637 | 4208.295
Replacing 25 samples.
1526.402 | 4207.519
Replacing 25 samples.
1520.246 | 4206.782
Replacing 25 samples.
1513.741 | 4207.050
Replacing 25 samples.
1507.940 | 4206.194
Replacing 25 samples.
1502.563 | 4205.630
Replacing 25 samples.
1496.717 | 4205.229
Replacing 25 samples.
1493.035 | 

1656.466 | 4067.096
Replacing 25 samples.
1657.791 | 4066.691
Replacing 25 samples.
1656.466 | 4067.096
Replacing 25 samples.
1657.791 | 4066.691
Replacing 25 samples.
1656.466 | 4067.096
Replacing 25 samples.
1657.791 | 4066.691
Replacing 25 samples.
1656.466 | 4067.096
Replacing 25 samples.
1657.791 | 4066.691
Replacing 25 samples.
1656.466 | 4067.096
Replacing 25 samples.
1657.791 | 4066.691
Replacing 25 samples.
1656.466 | 4067.096
Replacing 25 samples.
1657.791 | 4066.691
Replacing 25 samples.
1656.466 | 4067.096
Replacing 25 samples.


KeyboardInterrupt: 